In [1]:
from casacore.tables import table
import numpy as np

Summary: TIME is in JD or something. ANTENNA2 is the reference antenna. SCAN_NUMBER is always 1. FLAG needs to be rewritten.

In [49]:
t = table('/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/00.gcal')

Successful readonly open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/test-gain/00.gcal: 13 columns, 8012 rows


In [4]:
t.colnames()

['TIME',
 'FIELD_ID',
 'SPECTRAL_WINDOW_ID',
 'ANTENNA1',
 'ANTENNA2',
 'INTERVAL',
 'SCAN_NUMBER',
 'OBSERVATION_ID',
 'CPARAM',
 'PARAMERR',
 'FLAG',
 'SNR',
 'WEIGHT']

In [31]:
t.getcol('FLAG').shape

(8012, 130, 2)

In [45]:
t.putcol('FLAG', np.full_like(t.getcol('FLAG'), False))

In [35]:
t.putcol('CPARAM', np.full_like(t.getcol('CPARAM'), 2+0j))

In [51]:
t.getcol('CPARAM').shape

(8012, 130, 2)

In [52]:
t.close()

Now let's see what it does to a MS...I tested it out.

So applycal divides the data by the appropriate CPARAM products. If the FLAG column is TRUE, then it doesn't do anything with that antenna-time-channel solution.

In [39]:
from casatasks import applycal

In [47]:
%time applycal(vis='/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/00.ms', gaintable='/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/00.gcal', flagbackup=False, applymode='calonly')

CPU times: user 2min 29s, sys: 2min 21s, total: 4min 51s
Wall time: 10min 18s


In [69]:
phase = np.random.normal(scale=0.1, size=11)
gain = np.exp(-1j * (2 * np.pi) * phase/360)
gain_repeat = np.repeat(gain[:, np.newaxis], 3 * 2, axis=1).reshape(11, 3, 2)
gain_repeat.shape

(11, 3, 2)

In [64]:
gain[:np.newaxis,np.newaxis].shape

(1, 1, 11)

In [75]:
def gauss_phase_per_ant_time(bcal, rms_deg):
    with table(bcal, readonly=False) as t:
        t.putcol('FLAG', np.full_like(t.getcol('FLAG'), False))
        shape = t.getcol('CPARAM').shape
        phase = np.random.normal(scale=rms_deg, size=shape[0])
        gain = np.exp(-1j * (2 * np.pi) * phase/360)
        gain_repeat = np.repeat(gain[:,np.newaxis,np.newaxis], shape[1] * shape[2]).reshape(shape)
        t.putcol('CPARAM', gain_repeat)

In [76]:
import glob

In [77]:
l = glob.glob('/lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/*gcal')

In [78]:
for c in l:
    gauss_phase_per_ant_time(c, 0.1)

Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/06.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/10.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/12.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/04.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/09.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/02.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/13.ms.gcal: 13 columns, 8012 rows
Successful read/write open of defa

In [83]:
tt = table('/lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/07.ms.gcal')

Successful readonly open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/07.ms.gcal: 13 columns, 8012 rows


In [84]:
np.unique(tt.getcol('FLAG'))

array([False])

In [86]:
np.abs(tt.getcol('CPARAM'))

array([[[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       ...,

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]]], dtype=float32)

In [87]:
tt.close()